In [1]:
import pandas as pd
import numpy as np
import pickle as pkl

c:\Users\Khushi\anaconda3\Lib\site-packages\pandas\core\arrays\masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


In [2]:
from sklearn.model_selection import train_test_split
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import MinMaxScaler
from sklearn.feature_selection import SelectKBest,chi2
from sklearn.tree import DecisionTreeClassifier
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score
from sklearn.model_selection import cross_val_score

In [3]:
df = pd.read_csv('titanic.csv')

In [4]:
df.sample(5)

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
604,605,1,1,"Homer, Mr. Harry (""Mr E Haven"")",male,35.0,0,0,111426,26.5500,NaN,C
132,133,0,3,"Robins, Mrs. Alexander A (Grace Charity Laury)",female,47.0,1,0,A/5. 3337,14.5000,NaN,S
590,591,0,3,"Rintamaki, Mr. Matti",male,35.0,0,0,STON/O 2. 3101273,7.1250,NaN,S
49,50,0,3,"Arnold-Franchi, Mrs. Josef (Josefine Franchi)",female,18.0,1,0,349237,17.8000,NaN,S
332,333,0,1,"Graham, Mr. George Edward",male,38.0,0,1,PC 17582,153.4625,C91,S


In [5]:
df.drop(columns=['PassengerId','Name','Ticket','Cabin'],inplace=True)

In [6]:
df.sample(5)

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
692,1,3,male,NaN,0,0,56.4958,S
185,0,1,male,NaN,0,0,50.0000,S
537,1,1,female,30.0,0,0,106.4250,C
221,0,2,male,27.0,0,0,13.0000,S
83,0,1,male,28.0,0,0,47.1000,S


In [7]:
x_train, x_test, y_train, y_test = train_test_split((df.drop(columns=['Survived'])),
                                                    df['Survived'],
                                                    test_size=0.1,
                                                    random_state=42)

In [8]:
x_train.sample(5)

,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
220,3,male,16.0,0,0,8.0500,S
24,3,female,8.0,3,1,21.0750,S
749,3,male,31.0,0,0,7.7500,Q
46,3,male,NaN,1,0,15.5000,Q
727,3,female,NaN,0,0,7.7375,Q


In [9]:
y_train.sample(5)

760    0
624    0
577    1
829    1
770    0
Name: Survived, dtype: int64

In [10]:
df.isnull().sum()

Survived      0
Pclass        0
Sex           0
Age         177
SibSp         0
Parch         0
Fare          0
Embarked      2
dtype: int64

In [11]:
trf1 = ColumnTransformer([
    ('age',SimpleImputer(),[2]),
    ('embarked',SimpleImputer(strategy='most_frequent'),[-1])
],remainder='passthrough')

In [12]:
trf2 = ColumnTransformer([
    ('ohe',OneHotEncoder(drop='first',sparse_output=False,handle_unknown='ignore'),[1,-1])
],remainder='passthrough')

In [13]:
trf3 = ColumnTransformer([
    ('norm',MinMaxScaler(),slice(0,8))
])

In [14]:
trf4 = SelectKBest(score_func=chi2,k=8)

In [15]:
trf5 = DecisionTreeClassifier()

In [16]:
pipe = Pipeline([
    ('trf1',trf1),
    ('trf2',trf2),
    ('trf3',trf3),
    ('trf4',trf4),
    ('trf5',trf5),
])

In [17]:
pipe.fit(x_train,y_train)

Pipeline(steps=[('trf1',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('age', SimpleImputer(), [2]),
                                                 ('embarked',
                                                  SimpleImputer(strategy='most_frequent'),
                                                  [-1])])),
                ('trf2',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('ohe',
                                                  OneHotEncoder(drop='first',
                                                                handle_unknown='ignore',
                                                                sparse_output=False),
                                                  [1, -1])])),
                ('trf3',
                 ColumnTransformer(transformers=[('norm', MinMaxScaler(),
                                                  slice(0, 8, None))])),
                ('trf4',
                 SelectKBest(k=8,
                             score_func=<function chi2 at 0x000002D9F0AC2C00>)),
                ('trf5', DecisionTreeClassifier())])

In [18]:
pipe.named_steps

{'trf1': ColumnTransformer(remainder='passthrough',
                   transformers=[('age', SimpleImputer(), [2]),
                                 ('embarked',
                                  SimpleImputer(strategy='most_frequent'),
                                  [-1])]),
 'trf2': ColumnTransformer(remainder='passthrough',
                   transformers=[('ohe',
                                  OneHotEncoder(drop='first',
                                                handle_unknown='ignore',
                                                sparse_output=False),
                                  [1, -1])]),
 'trf3': ColumnTransformer(transformers=[('norm', MinMaxScaler(), slice(0, 8, None))]),
 'trf4': SelectKBest(k=8, score_func=<function chi2 at 0x000002D9F0AC2C00>),
 'trf5': DecisionTreeClassifier()}

In [19]:
y_pred = pipe.predict(x_test)

c:\Users\Khushi\anaconda3\Lib\site-packages\sklearn\preprocessing\_encoders.py:246: UserWarning: Found unknown categories in columns [1] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(


In [20]:
y_pred

array([1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0,
       1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 1, 1, 0, 1, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 1, 1, 1,
       0, 0], dtype=int64)

In [21]:
acc = accuracy_score(y_pred,y_test)

In [22]:
acc

0.6444444444444445

In [23]:
cvs = cross_val_score(pipe,x_train,y_train,cv=5,scoring='accuracy')

c:\Users\Khushi\anaconda3\Lib\site-packages\sklearn\preprocessing\_encoders.py:246: UserWarning: Found unknown categories in columns [1] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(
c:\Users\Khushi\anaconda3\Lib\site-packages\sklearn\preprocessing\_encoders.py:246: UserWarning: Found unknown categories in columns [1] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(
c:\Users\Khushi\anaconda3\Lib\site-packages\sklearn\preprocessing\_encoders.py:246: UserWarning: Found unknown categories in columns [1] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(
c:\Users\Khushi\anaconda3\Lib\site-packages\sklearn\preprocessing\_encoders.py:246: UserWarning: Found unknown categories in columns [1] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(
c:\Users\Khushi\anaconda3\Lib\site-packages\sklearn\preprocessing\_encoders.py:246: UserWarning:

In [24]:
cvs.mean()

0.6354813664596274

In [25]:
cvs.max()

0.6625

In [26]:
pkl.dump(pipe,open('titanic_model.plk','wb'))